In [1]:
import json
import os
import glob

In [2]:
imgNamesPath = '../../dataset/image_names_split'

numberSplitPath = '../../dataset/object_number_split'
typeSplitPath = '../../dataset/object_type_split'

In [3]:
valQuestionsObject = json.load(open('../../../CLEVR_v1.0/questions/CLEVR_val_questions.json'))['questions']
print(len(valQuestionsObject))

149991


In [4]:
valScenesObject = json.load(open('../../../CLEVR_v1.0/scenes/CLEVR_val_scenes.json'))['scenes']
print(len(valScenesObject))

15000


In [5]:
!ls

generate_dataset.ipynb	split_dataset.py


In [6]:
def is_number(value):
    if isinstance(value, (int, float, complex)):
        return True
    elif isinstance(value, str) and value.isnumeric():
        return True
    else:
        return False

def ansType(ans):
    if ans.lower() == 'yes' or ans.lower()== 'no':
        return 'yes-no'
    elif is_number(ans):
        return 'number'
    elif ans in ["cube", "sphere", "cylinder"] :
        return 'shape'
    elif ans in  ["small", "large"]:
        return 'size'
    elif ans in [ "left", "right", "behind", "front"]:
        return 'relation'
    elif ans in [ "gray", "red", "blue", "green", "brown", "purple", "cyan", "yellow"]:
        return 'colour'
    elif ans in ["rubber", "metal"]:
        return 'material'
    
    return 'none'

In [7]:
for targetNumberOfObjects in range(3,11):
    dataset = []
    yn = 0
    others = 0
    idx = -1
    for question in valQuestionsObject:
        imageIndex = question['image_index']
        numberOfObjectsInImage = len(valScenesObject[imageIndex]['objects'])
        answerType = ansType(question['answer'])

        if targetNumberOfObjects != numberOfObjectsInImage or answerType in ['none', 'size', 'material']:
            continue

        if idx != imageIndex:
            idx = imageIndex
            yn = 0
            others = 0

        if answerType == 'yes-no':
            yn += 1
            if yn > 2:
                continue
        else:
            others += 1
            if others > 3:
                continue

        dataset.append({
            'image_filename': question['image_filename'],
            'question_index': question['question_index'],
            'answer_type': answerType,
            'question': question['question'],
            'ground_truth': question['answer'],
            'model_generated_answer': '',
            'consistent': '',
        })

        # print(dataset)
        # break
    file = open(f'../../dataset/object_number_split/val_num_{targetNumberOfObjects}.json', 'w')
    json.dump(dataset, file)
    file.close()

In [8]:
for type in ['inter', 'intra']:
    dataset = []
    yn = 0
    others = 0
    idx = -1

    for question in valQuestionsObject:
        imageIndex = question['image_index']
        answerType = ansType(question['answer'])
        split = ''

        objectsInImage = valScenesObject[imageIndex]['objects']
        objectsInSet = set()

        for obj in objectsInImage:
            objectsInSet.add(obj['shape'])

        if len(objectsInSet) > 1:
            split = 'inter'
        else:
            split = 'intra'

        if type != split or answerType in ['none', 'size', 'material']:
            continue

        if idx != imageIndex:
            idx = imageIndex
            yn = 0
            others = 0

        if answerType == 'yes-no':
            yn += 1
            if yn > 2:
                continue
        else:
            others += 1
            if others > 3:
                continue

        dataset.append({
            'image_filename': question['image_filename'],
            'question_index': question['question_index'],
            'answer_type': answerType,
            'question': question['question'],
            'ground_truth': question['answer'],
            'model_generated_answer': '',
            'consistent': '',
        })

    file = open(f'../../dataset/object_type_split/val_type_{type}.json', 'w')
    json.dump(dataset, file)
    file.close()

In [9]:
# validating
path = '../../dataset/object_number_split'
files = glob.glob(os.path.join(path, '*.json'), recursive=True)
sum1 = 0

for f in files:
    o = json.load(open(f))
    sum1 += len(o)

print(sum1)

####################
path = '../../dataset/object_type_split'
files = glob.glob(os.path.join(path, '*.json'), recursive=True)
sum2 = 0

for f in files:
    o = json.load(open(f))
    sum2 += len(o)

print(sum2)

72223
72223
